In [59]:
import requests
import base64
import datetime 
from urllib.parse import urlencode
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
from functools import reduce
import operator

In [60]:
client_id = '45844a803da249adae64a637787bfd7e'
client_secret = 'dc6e854038474d0ab8ed4faacc9d3080'

In [83]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

In [70]:
spotify = SpotifyAPI(client_id, client_secret)

In [71]:
spotify.perform_auth()

True

In [72]:
spotify.access_token

'BQBLuVuTwRclTFBMDv-SAFIJKu_S4aNIrmLyY9OCb_KxpIwRID9iDLSmBsgMFlbk7Vm7MBTVDIEbpkVeNSU'

In [73]:
access_token = spotify.access_token
access_token

'BQBLuVuTwRclTFBMDv-SAFIJKu_S4aNIrmLyY9OCb_KxpIwRID9iDLSmBsgMFlbk7Vm7MBTVDIEbpkVeNSU'

In [84]:
headers = {
    "Authorization": f"Bearer {access_token}"
}
endpoint = f"https://api.spotify.com/v1/playlists/1MCXV57pNJkJy4EQPLBzn0/tracks" 
#?offset=0&limit=100&additional"
data = urlencode({"additional_types": "track", "episode": "track"})
print(data)

lookup_url = f"{endpoint}?{data}"
print(lookup_url)
r = requests.get(lookup_url, headers=headers)
print(r.status_code)

additional_types=track&episode=track
https://api.spotify.com/v1/playlists/1MCXV57pNJkJy4EQPLBzn0/tracks?additional_types=track&episode=track
200


In [96]:
r

<Response [200]>

In [97]:
df = r.json()

In [124]:
df

{'href': 'https://api.spotify.com/v1/playlists/1MCXV57pNJkJy4EQPLBzn0/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2020-12-10T15:59:07Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/1195145834'},
    'href': 'https://api.spotify.com/v1/users/1195145834',
    'id': '1195145834',
    'type': 'user',
    'uri': 'spotify:user:1195145834'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0epOFNiUfyON9EYx7Tpr6V'},
       'href': 'https://api.spotify.com/v1/artists/0epOFNiUfyON9EYx7Tpr6V',
       'id': '0epOFNiUfyON9EYx7Tpr6V',
       'name': 'The Strokes',
       'type': 'artist',
       'uri': 'spotify:artist:0epOFNiUfyON9EYx7Tpr6V'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      '

In [175]:
nueva  = df.get("items")
def getFromDict(diccionario,objeto):
    return reduce(operator.getitem,objeto,diccionario)
def places_df(frame):
    nombre = ["track","name"]
    x = []
    for diccionario in frame:
        lista = {}
        lista["name"] = getFromDict(diccionario,nombre)
        x.append(lista)
    return x
places_df(nueva)

[{'name': 'The Adults Are Talking'},
 {'name': 'rue'},
 {'name': 'Can I Call You Tonight?'},
 {'name': 'Double Negative (Skeleton Milkshake)'},
 {'name': 'Where Is My Mind?'},
 {'name': 'In Love and Alone'},
 {'name': 'Spanish Bombs - Remastered'},
 {'name': 'Wasting Time'},
 {'name': "Devil's Advocate"},
 {'name': 'Waiting For You'},
 {'name': 'Hello, Goodbye - Remastered 2015'},
 {'name': 'Time is Running Out'},
 {'name': 'Tie Me Up'},
 {'name': 'Heathers'},
 {'name': 'Bag It Up'},
 {'name': 'Bad Decisions'},
 {'name': 'Sweater Weather'},
 {'name': 'No me gustas, te quiero'},
 {'name': 'Just Yesterday'},
 {'name': 'Easy'}]

In [176]:
df_final = pd.DataFrame(places_df(nueva))

In [177]:
df_final

,name
0,The Adults Are Talking
1,rue
2,Can I Call You Tonight?
3,Double Negative (Skeleton Milkshake)
4,Where Is My Mind?
5,In Love and Alone
6,Spanish Bombs - Remastered
7,Wasting Time
8,Devil's Advocate
9,Waiting For You


In [194]:
df_final.to_csv('../data/playlists.csv')

In [181]:
nueva  = df.get("items")
def getFromDict(diccionario,objeto):
    return reduce(operator.getitem,objeto,diccionario)
def places_df(frame):
    nombre = ["track","name"]
    artista = ["track", "artists"]
    x = []
    for diccionario in frame:
        lista = {}
        lista["name"] = getFromDict(diccionario,nombre)
        lista["artists"] = getFromDict(diccionario,artista)
        x.append(lista)
    return x
places_df(nueva)

[{'name': 'The Adults Are Talking',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0epOFNiUfyON9EYx7Tpr6V'},
    'href': 'https://api.spotify.com/v1/artists/0epOFNiUfyON9EYx7Tpr6V',
    'id': '0epOFNiUfyON9EYx7Tpr6V',
    'name': 'The Strokes',
    'type': 'artist',
    'uri': 'spotify:artist:0epOFNiUfyON9EYx7Tpr6V'}]},
 {'name': 'rue',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3uwAm6vQy7kWPS2bciKWx9'},
    'href': 'https://api.spotify.com/v1/artists/3uwAm6vQy7kWPS2bciKWx9',
    'id': '3uwAm6vQy7kWPS2bciKWx9',
    'name': 'girl in red',
    'type': 'artist',
    'uri': 'spotify:artist:3uwAm6vQy7kWPS2bciKWx9'}]},
 {'name': 'Can I Call You Tonight?',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6eJa3zG1QZLRB3xgRuyxbm'},
    'href': 'https://api.spotify.com/v1/artists/6eJa3zG1QZLRB3xgRuyxbm',
    'id': '6eJa3zG1QZLRB3xgRuyxbm',
    'name': 'Dayglow',
    'type': 'artist',
    'uri': 'spotify:a

In [193]:
otro  = places_df(nueva).get("name")
def getFromDict(diccionario,objeto):
    return reduce(operator.getitem,objeto,diccionario)
def places_df(frame):
    nombre = ["name", "name"]
    artista = ["artists", "names"]
    x = []
    for diccionario in frame:
        lista = {}
        lista["names"] = getFromDict(diccionario,nombre)
        lista["artists"] = getFromDict(diccionario,artista)
        x.append(lista)
    return x
places_df(nueva)

KeyError: 'name'

In [182]:
df_final_2 = pd.DataFrame(places_df(nueva))

In [183]:
df_final_2

,name,artists
0,The Adults Are Talking,[{'external_urls': {'spotify': 'https://open.s...
1,rue,[{'external_urls': {'spotify': 'https://open.s...
2,Can I Call You Tonight?,[{'external_urls': {'spotify': 'https://open.s...
3,Double Negative (Skeleton Milkshake),[{'external_urls': {'spotify': 'https://open.s...
4,Where Is My Mind?,[{'external_urls': {'spotify': 'https://open.s...
5,In Love and Alone,[{'external_urls': {'spotify': 'https://open.s...
6,Spanish Bombs - Remastered,[{'external_urls': {'spotify': 'https://open.s...
7,Wasting Time,[{'external_urls': {'spotify': 'https://open.s...
8,Devil's Advocate,[{'external_urls': {'spotify': 'https://open.s...
9,Waiting For You,[{'external_urls': {'spotify': 'https://open.s...


In [184]:
list(r.json())

['href', 'items', 'limit', 'next', 'offset', 'previous', 'total']

In [ ]:
<iframe src="https://open.spotify.com/embed/playlist/1MCXV57pNJkJy4EQPLBzn0" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>